# M102 - ¿Qué es la clasificación?

Por [Alan Badillo Salas](mailto:alan@nomadacode.com)

La **minería de datos** tiene el objetivo de enseñarle a la máquina a tomar decisiones a partir de los datos. Esta se compone de un conjunto de algorimos que involucran la estadística, ingeniería de datos, optimización y cómputo científico (ciencia de datos, aprendizaje automático, aprendizaje profundo, etc).

Un conjuntos de datos (*datasets*) se compone de muestras (*samples*) a las que se les pueden extraer características (*features*).

El algorimo general de la minería de datos involucra los siguientes pasos:

1. Construir un conjunto de datos (*dataset*) describiendo parte del mundo real. Aquí disponemos de muestras, que se refieren a las observaciones que realizamos como un libro, una persona, un animal, etc. También extraemos características, que se refiere a mediciones sobre las muestras, por ejemplo, el tamaño, la frecuencia de una palabra específica, el número de patas de un animal, estas se conocen generalmente como variables, columnas, atributos, covariantes, etc.
2. Entrenar al algoritmo basado en el conjunto de datos, ajustando los parámetros del algoritmo, para permitirle al algoritmo saber cómo tomar decisiones a partir de los datos.

Veamos un ejemplo de cómo nos gustaría que la máquina sea capaz de categorizar personas dándole algunas muestras de personas de estatura bajas y altas.

| Persona | Estatura | Etiqueta |
|---------|----------|----------|
| 1       | 155cm    | Baja     |
| 2       | 155cm    | Baja     |
| 3       | 175cm    | Alta     |
| 4       | 185cm    | Alta     |



Generador de un conjunto de datos sobre compras de productos

In [3]:
import random

features = ["Leche", "Queso", "Pan", "Manzana", "Huevo"]

dataset = []

n_samples = 100
n_features = len(features) # 5

for i in range(n_samples):

    sample = [0, 0, 0, 0, 0]

    dataset.append(sample)

    # Probabilidad de comprar Leche
    if random.random() < 0.25:
        sample[0] = 1

        # Dado que compró Leche

        # Probabilidad de comprar Queso
        if random.random() < 0.5:
            sample[1] = 1

        # Probabilidad de comprar Pan
        if random.random() < 0.3:
            sample[2] = 1

            # Dado que compró Pan:

            # Probabilidad de comprar Huevo
            if random.random() < 0.6:
                sample[4] = 1
        else:

            # Dado que no compró Pan

            # Probabilidad de comprar Manzana
            if random.random() < 0.8:
                sample[3] = 1
    else:

        # Dado que no compró Leche

        # Probabilidad de comprar Queso
        if random.random() < 0.2:
            sample[1] = 1

        # Probabilidad de comprar Pan
        if random.random() < 0.1:
            sample[2] = 1

            # Dado que compró Pan:

            # Probabilidad de comprar Huevo
            if random.random() < 0.6:
                sample[4] = 1
        else:

            # Dado que no compró Pan

            # Probabilidad de comprar Manzana
            if random.random() < 0.8:
                sample[3] = 1

print(dataset[:5])

[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0]]


In [15]:
def print_sample(sample):
    print("Compró", end=": ")
    count = 0
    for index_sample in range(n_features):
        label = features[index_sample]
        if sample[index_sample] == 1:
            print(label, end=" ")
            count += 1
    if count == 0:
        print("<NADA>", end="")
    print()

def print_dataset(dataset):
    for sample in dataset:
        print_sample(sample)

print_dataset(dataset[50:55])

Compró: Manzana 
Compró: <NADA>
Compró: Leche Queso Manzana 
Compró: Pan Huevo 
Compró: Manzana 


In [23]:
from collections import defaultdict

num_ocurrences = defaultdict(int)
valid_rules = defaultdict(int)

num_ocurrences, valid_rules

(defaultdict(int, {}), defaultdict(int, {}))

In [24]:
for sample in dataset:
    for premise_index in range(n_features):
        if sample[premise_index] == 1:
            num_ocurrences[premise_index] += 1
            for conclusion_index in range(n_features):
                if premise_index == conclusion_index:
                    continue
                if sample[conclusion_index] == 1:
                    valid_rules[(premise_index, conclusion_index)] += 1

num_ocurrences, valid_rules

(defaultdict(int, {3: 58, 1: 29, 0: 28, 2: 22, 4: 14}),
 defaultdict(int,
             {(0, 1): 18,
              (0, 3): 15,
              (1, 0): 18,
              (1, 3): 16,
              (3, 0): 15,
              (3, 1): 16,
              (2, 4): 14,
              (4, 2): 14,
              (1, 2): 11,
              (2, 1): 11,
              (0, 2): 12,
              (2, 0): 12,
              (0, 4): 8,
              (4, 0): 8,
              (1, 4): 7,
              (4, 1): 7}))

In [31]:
from operator import itemgetter

for premise_index, count in sorted(num_ocurrences.items(), key=itemgetter(0)):
    print("COMPRÓ {} ({} veces)".format(features[premise_index], count))

COMPRÓ Leche (28 veces)
COMPRÓ Queso (29 veces)
COMPRÓ Pan (22 veces)
COMPRÓ Manzana (58 veces)
COMPRÓ Huevo (14 veces)


In [41]:
for (premise_index, conclusion_index), count in sorted(valid_rules.items(), key=itemgetter(1), reverse=True):
    confidence = count / num_ocurrences[premise_index]
    print("SI {} -> {} ({} veces)".format(features[premise_index], features[conclusion_index], num_ocurrences[premise_index]))
    print("CONFIANZA: {:.1f}%".format(confidence * 100))
    print("SOPORTE: {}".format(count))

SI Leche -> Queso (28 veces)
CONFIANZA: 64.3%
SOPORTE: 18
SI Queso -> Leche (29 veces)
CONFIANZA: 62.1%
SOPORTE: 18
SI Queso -> Manzana (29 veces)
CONFIANZA: 55.2%
SOPORTE: 16
SI Manzana -> Queso (58 veces)
CONFIANZA: 27.6%
SOPORTE: 16
SI Leche -> Manzana (28 veces)
CONFIANZA: 53.6%
SOPORTE: 15
SI Manzana -> Leche (58 veces)
CONFIANZA: 25.9%
SOPORTE: 15
SI Pan -> Huevo (22 veces)
CONFIANZA: 63.6%
SOPORTE: 14
SI Huevo -> Pan (14 veces)
CONFIANZA: 100.0%
SOPORTE: 14
SI Leche -> Pan (28 veces)
CONFIANZA: 42.9%
SOPORTE: 12
SI Pan -> Leche (22 veces)
CONFIANZA: 54.5%
SOPORTE: 12
SI Queso -> Pan (29 veces)
CONFIANZA: 37.9%
SOPORTE: 11
SI Pan -> Queso (22 veces)
CONFIANZA: 50.0%
SOPORTE: 11
SI Leche -> Huevo (28 veces)
CONFIANZA: 28.6%
SOPORTE: 8
SI Huevo -> Leche (14 veces)
CONFIANZA: 57.1%
SOPORTE: 8
SI Queso -> Huevo (29 veces)
CONFIANZA: 24.1%
SOPORTE: 7
SI Huevo -> Queso (14 veces)
CONFIANZA: 50.0%
SOPORTE: 7
